<center>
$$ \huge \textbf{FundusD121: DenseNet121 Model} $$ 🔬👁️  
</center>

**Purpose:**  
- 💡 Classifies fundus images as 'Normal' or 'AbNormal' for Alzheimer's detection.

**Model Overview:**  
- 🔑 **Backbone:** DenseNet121 — A dense convolutional network with 121 layers, improving feature reuse.

**Techniques Used:**  
- 🔄 **Transfer Learning:** Using pre-trained ImageNet weights.  
- ⚖️ **Binary Cross-Entropy Loss** for binary classification.  
- 🧑‍🏫 **Adam Optimizer** with a learning rate of **0.0001**.  
- ⚖️ **Class Weight Balancing** for addressing imbalanced data.  
- ⏱️ **Callbacks:** Early stopping & ReduceLROnPlateau to speed up training.

In [ ]:
import tensorflow as tf
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from sklearn.utils.class_weight import compute_class_weight

# ✅ **Mount Google Drive**
from google.colab import drive
drive.mount('/content/drive')

# ✅ **Dataset Path**
dataset_path = "/content/drive/MyDrive/Fundus-ModelData"

train_dir = os.path.join(dataset_path, "Test-Imgs")
test_dir = os.path.join(dataset_path, "Train-Imgs")

# ✅ **Hyperparameters**
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 50
LEARNING_RATE = 0.0001

# ✅ **Data Augmentation**
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2],
    validation_split=0.2
)

test_datagen = ImageDataGenerator(rescale=1.0 / 255)

# ✅ **Load Data**
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="binary",
    subset="training"
)

val_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="binary",
    subset="validation"
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="binary",
    shuffle=False
)

# ✅ **Handle Class Imbalance (Class Weights)**
class_labels = np.array(train_generator.classes)
class_weights = compute_class_weight(
    class_weight="balanced",
    classes=np.unique(class_labels),
    y=class_labels
)
class_weight_dict = {i: class_weights[i] for i in range(len(class_weights))}

print(f"Class Weights: {class_weight_dict}")

# ✅ **Load DenseNet121 (Pretrained)**
base_model = DenseNet121(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Freeze base model

# ✅ **Add Custom Layers**
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
x = Dense(256, activation="relu")(x)
x = Dropout(0.3)(x)
output = Dense(1, activation="sigmoid")(x)

# ✅ **Compile Model**
model = Model(inputs=base_model.input, outputs=output)
model.compile(
    optimizer=Adam(learning_rate=LEARNING_RATE),
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

# ✅ **Model Saving Paths**
best_model_path = "/content/drive/MyDrive/draft_models/densenet121_best.h5"
final_model_path = "/content/drive/MyDrive/draft_models/densenet121_final.h5"
saved_model_dir = "/content/drive/MyDrive/draft_models/densenet121_saved_model.h5"

# ✅ **Callbacks**
checkpoint = ModelCheckpoint(
    best_model_path, monitor="val_loss", save_best_only=True, mode="min", verbose=1
)
reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3, verbose=1)
early_stopping = EarlyStopping(monitor="val_loss", patience=7, restore_best_weights=True)

# ✅ **Train the Model**
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=EPOCHS,
    class_weight=class_weight_dict,
    callbacks=[checkpoint, reduce_lr, early_stopping]
)

# ✅ **Save the Final Model**
model.save(final_model_path)
model.save(saved_model_dir)  # No need to specify save_format
print(f"Model saved at: {final_model_path} and {saved_model_dir}")

# ✅ **Plot Training Curves**
def plot_training_curves(history):
    plt.figure(figsize=(12, 5))

    # Accuracy Plot
    plt.subplot(1, 2, 1)
    plt.plot(history.history["accuracy"], label="Train Accuracy")
    plt.plot(history.history["val_accuracy"], label="Validation Accuracy")
    plt.xlabel("Epochs")
    plt.ylabel("Accuracy")
    plt.legend()
    plt.title("Training & Validation Accuracy")

    # Loss Plot
    plt.subplot(1, 2, 2)
    plt.plot(history.history["loss"], label="Train Loss")
    plt.plot(history.history["val_loss"], label="Validation Loss")
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.legend()
    plt.title("Training & Validation Loss")

    plt.show()

plot_training_curves(history)

**Prediction:**
- 🧑‍💻 Predict the 'Normal' or 'AbNormal' status of new fundus images.

**Steps:**
1. 📂 Load the model that has been trained.  
2. 🔄 Preprocess the new images (resize, normalize).  
3. 📊 Use the model to predict the class of the images.  
4. 🧐 Show the prediction results with confidence scores.

In [ ]:
import tensorflow as tf
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from fpdf import FPDF
from google.colab import drive

# 🚀 Mount Google Drive
drive.mount('/content/drive')

# 🔍 Define paths
model_path = "/content/drive/MyDrive/Fundus-ModelData/Saved-Models/4-FundusD121-DenseNet121/FundusD121.h5"
test_folder = "/content/drive/MyDrive/Fundus-ModelData/Test-Imgs"
output_pdf_path = "/content/drive/MyDrive/Fundus-ModelData/Saved-Models/4-FundusD121-DenseNet121/FundusD121"

# 🧠 Load Trained Model
model = tf.keras.models.load_model(model_path)

# 🖼 Preprocess Image
def preprocess_image(image_path):
    img = load_img(image_path, target_size=(224, 224))  # Resize
    img_array = img_to_array(img) / 255.0  # Normalize
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    return img_array

# 📂 Process Images & Predict
results = []
class_counts = {}
confidence_scores = {}
threshold = 0.5  # Sigmoid activation threshold

for subfolder in os.listdir(test_folder):
    subfolder_path = os.path.join(test_folder, subfolder)

    if os.path.isdir(subfolder_path):  # Ensure it's a folder
        class_counts[subfolder] = {"Normal": 0, "Abnormal": 0}
        confidence_scores[subfolder] = []

        for image_name in os.listdir(subfolder_path):
            image_path = os.path.join(subfolder_path, image_name)

            # Predict class
            img_array = preprocess_image(image_path)
            prediction = model.predict(img_array)[0][0]
            predicted_class = "Abnormal" if prediction > threshold else "Normal"
            confidence_score = round(prediction, 4)

            # Track counts & confidence
            class_counts[subfolder][predicted_class] += 1
            confidence_scores[subfolder].append(confidence_score)

            # Store results
            results.append([image_name, subfolder, predicted_class, confidence_score])

# 📝 Convert results to DataFrame
df = pd.DataFrame(results, columns=["Image Name", "Folder", "Predicted Class", "Confidence Score"])

# 🎨 Generate Data Visualizations
fig, axes = plt.subplots(1, len(class_counts), figsize=(10, 5))
for i, (folder, counts) in enumerate(class_counts.items()):
    labels = counts.keys()
    sizes = counts.values()
    axes[i].pie(sizes, labels=labels, autopct="%1.1f%%", startangle=90, colors=["lightblue", "salmon"])
    axes[i].set_title(f"Class Distribution in {folder}")
plt.savefig("/content/class_distribution.png")

# 📊 Average Confidence per Folder
avg_confidence = {folder: np.mean(scores) for folder, scores in confidence_scores.items()}
plt.figure(figsize=(6, 4))
plt.bar(avg_confidence.keys(), avg_confidence.values(), color="skyblue")
plt.xlabel("Folders")
plt.ylabel("Average Confidence Score")
plt.title("Average Confidence Score per Subfolder")
plt.savefig("/content/avg_confidence.png")

!pip install fpdf

# 📄 Generate PDF Report
class PDF(FPDF):
    def header(self):
        self.set_font("Times", "B", 16)
        self.cell(275, 10, "Fundus Image Classification Report", ln=True, align="C")

    def footer(self):
        self.set_y(-15)
        self.set_font("Times", "I", 10)
        self.cell(0, 10, f"Page {self.page_no()}", align="C")

# 📑 Create PDF
pdf = PDF(orientation="L")
pdf.set_auto_page_break(auto=True, margin=15)
pdf.add_page()
pdf.set_font("Times", size=12)

# 🏷 Report Header
pdf.set_font("Times", "B", 14)
pdf.cell(0, 10, "Model Details", ln=True)
pdf.set_font("Times", size=12)
pdf.cell(0, 10, f"Model Name: FundusD121", ln=True)
#pdf.cell(0, 10, f"Model Path: {model_path}", ln=True)
pdf.cell(0, 10, "Author: Danish A. G.", ln=True)
pdf.ln(10)

# 📋 Results Table
pdf.set_font("Times", "B", 12)
pdf.cell(140, 10, "Image Name", 1)
pdf.cell(40, 10, "Folder", 1)
pdf.cell(40, 10, "Predicted Class", 1)
pdf.cell(50, 10, "Confidence Score", 1)
pdf.ln()

pdf.set_font("Times", size=12)
for idx, row in df.iterrows():
    pdf.cell(140, 10, row["Image Name"], 1)
    pdf.cell(40, 10, row["Folder"], 1)
    pdf.cell(40, 10, row["Predicted Class"], 1)
    pdf.cell(50, 10, str(row["Confidence Score"]), 1)
    pdf.ln()

# 📊 Insert Pie Charts
pdf.ln(10)
pdf.cell(0, 10, "Class Distribution", ln=True, align="L")
pdf.image("/content/class_distribution.png", x=10, w=250)

# 📉 Insert Average Confidence Score Chart
pdf.ln(10)
pdf.cell(0, 10, "Average Confidence Score per Subfolder", ln=True, align="L")
pdf.image("/content/avg_confidence.png", x=10, w=250)

# 💾 Save PDF to Drive
pdf.output(output_pdf_path)
print(f"✅ Report saved at: {output_pdf_path}")